## Import

In [ ]:
!pip install -r requirements.txt
!pip install deepchem
!pip install rdkit
!pip install pycm
!pip install pytorch-lightning wandb rdkit ogb
!pip install torch_geometric
!pip install optuna

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import deepchem as dc
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from rdkit import Chem
from rdkit.Chem import Draw, rdFingerprintGenerator, AllChem
from rdkit.Chem.Descriptors import MolWt, TPSA, NumHDonors, NumHAcceptors
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator

In [ ]:
df=pd.read_csv('content/Chemoinformatics_project/data/raw/sider.csv')

In [ ]:
df.head()

## Pre-processing

In [ ]:
from Classification.src.sider_preprocessing import sider_preprocessing

df_cleaned = sider_preprocessing(df)

## Featurizer

In [ ]:
from Classification.src.sider_featurizer import featurizer

df_final = featurizer(df=df_cleaned, mol_col='Molecule', fpSize=2048)

In [ ]:
df_final.columns

## ACP and feature selection

In [ ]:
from Classification.src.sider_pca import analyze_pca_variance
analyze_pca_variance(df_final)

Why **PCA is Not an Effective Strategy Here**
My analysis showed that using PCA for dimensionality reduction is not a good idea for this dataset.

- Low Variance Captured: The scree plot shows that the first principal component (PC1) only explains about 2.7% of the total variance. This is extremely low, as a useful PCA would capture a much larger portion.

- Information is Spread Out: This indicates that the information is already distributed very evenly across our original features. In other words, there is very little redundancy for PCA to compress.

- Significant Information Loss: If we were to use the top 10 components, we would only capture about 10-12% of the total information. Using PCA here would mean throwing away the vast majority of our data.

Note on Feature Scaling
For distance-based models like SVMs and Logistic Regression, applying StandardScaler is crucial. It normalizes the features, which is important for these algorithms as they are sensitive to the scale of the input data.

## Models

In [ ]:
X = df_final.iloc[:, 29:].copy()
y = df_final.iloc[:, 2:29]

X = X.select_dtypes(include=np.number)

# Apply VarianceThreshold to remove zero-variance features
selector = VarianceThreshold(threshold=0.0)
X_cleaned_array = selector.fit_transform(X)
X = pd.DataFrame(X_cleaned_array, columns=X.columns[selector.get_support()])

In [ ]:
# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=0)

# Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print(f"Data split: {len(X_train)} training samples, {len(X_test)} testing samples.")

In [ ]:
from Classification.sider_baseline_models.py import train_and_evaluate_models
train_and_evaluate_models(X_train, X_test, X_train_scaled, X_test_scaled, y_train, y_test)

## UMAP

In [ ]:
from sklearn.preprocessing import StandardScaler
import umap.umap_ as umap

# Scale the data once before creating the plots
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Calculate the UMAP projection
reducer = umap.UMAP(n_components=2, random_state=42)
embedding = reducer.fit_transform(X_scaled)

umap_df = pd.DataFrame(embedding, columns=['UMAP 1', 'UMAP 2'])

In [ ]:
# Plot interesting labels

from Classification.src.sider_umap import plot_umap

plot_umap(umap_df, y, 'Gastrointestinal disorders')
plot_umap(umap_df, y, 'Metabolism and nutrition disorders')
plot_umap(umap_df, y, 'Product issues')

1. High Performance: Gastrointestinal disorders
Observation: The plot is dominated by yellow dots (class 1), which confirms this is a highly prevalent side effect in the dataset.

Interpretation: Although the purple dots (class 0) do not form a single large cluster, they are not random. They group together in specific regions, indicating visual separability.

Conclusion: This clear, non-linear separation justifies why models like Random Forest were able to achieve high AUC-ROC scores (up to 0.76) for this label.

2. Low Performance: Metabolism and nutrition disorders
Observation: The plot shows a complete mix of yellow (class 1) and purple (class 0) dots. There is no discernible structure or clustering.

Interpretation: This visually represents a lack of a discriminating signal in the features. The Morgan fingerprints and descriptors do not seem to contain enough information to separate the classes for this specific label.

Conclusion: This directly explains why all models performed poorly, with AUC-ROC scores close to the random-guess baseline of 0.5.

3. Anomaly and Imbalance: Product issues
Observation: The UMAP plot is almost entirely purple (class 0), confirming that this is an extremely rare side effect. The few positive samples (yellow dots) are scattered without forming any clear cluster.

Interpretation: Given the extreme rarity of positive samples, the high AUC-ROC score of 0.7573 from XGBoost is highly suspect. There is no visual evidence of a pattern that would allow for such a strong classification.

Conclusion: The high score is likely a statistical artifact due to the very small number of positive samples in the test set. Therefore, the model's performance on this label should not be considered robust or reliable.

## SVM Optimization

Optimizing the SVM with a Non-Linear Kernel
My baseline results showed a clear pattern: tree-based models like Random Forest (Macro AUC-ROC of 0.66) significantly outperformed the linear models like Logistic Regression and the linear SVM (around 0.61).

This strongly suggested that the relationship between the molecular features and the side effects is non-linear.

The UMAP visualizations confirmed this hypothesis. For well-predicted labels, the classes formed distinct, non-linear clusters that couldn't be separated by a simple line. This visual evidence provided a strong reason to explore a more complex, non-linear SVM.

Optimization Strategy
Based on this, I decided to optimize an SVM using the RBF kernel to see if it could match the performance of the tree-based models.

Since SVMs are very slow to train, I used RandomizedSearchCV instead of an exhaustive GridSearchCV. This is a more efficient way to explore a wide range of hyperparameters (C and gamma) in a reasonable amount of time. If I had more time, a next step would be to try a more advanced technique like Bayesian Optimization.

Practical Challenge: Runtime
However, I ran into a major practical issue: the runtime.

Because this is a multi-label problem, the OneVsRestClassifier has to train 27 separate SVMs. With RandomizedSearchCV configured for 8 iterations and 3-fold cross-validation, this meant training a total of 648 models (27 * 8 * 3).

This process was too slow for a standard notebook environment. It's also worth noting that scikit-learn's SVM is CPU-bound, meaning a GPU would not have helped speed up this specific task. This experience showed that while powerful, SVMs can be impractical for hyperparameter tuning on complex, multi-label datasets like this one.

In [ ]:
from Classification.src.sider_svm_optimization import optimize_svm

# RandomizedSearchCV on non RBF SVM
best_svm_model = optimize_svm(X_train_scaled, y_train)

## GNN and Deep Learning

In [ ]:
# RUN THE TRAINING AND EVALUATION PIPELINE

from sklearn.model_selection import train_test_split
from torch_geometric.loader import DataLoader
import pytorch_lightning as pl
from Classification.src.sider_gnn import SIDERGraphDataset, MPNN_SIDER

sider_dataset = SIDERGraphDataset(root='./sider_pyg', df=df_cleaned)

# Split the dataset
# We split the list of graph objects, not X and y matrices.
train_data, test_data = train_test_split(
    sider_dataset, train_size=0.8, random_state=0, shuffle=True
)
train_data, val_data = train_test_split(
    train_data, train_size=0.9, random_state=42, shuffle=True # 10% of train for validation
)

# DataLoaders handle batching of graphs for the model
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Initialize and Train the Model
gnn_model = MPNN_SIDER(hidden_dim=64, out_dim=27, lr=0.001)

# PyTorch Lightning Trainer
trainer = pl.Trainer(max_epochs=50, log_every_n_steps=10)
trainer.fit(model=gnn_model, train_dataloaders=train_loader, val_dataloaders=val_loader)

# Evaluation
trainer.test(model=gnn_model, dataloaders=test_loader)

## Bayesian Optimization

In [ ]:
from sklearn.model_selection import train_test_split
from torch_geometric.loader import DataLoader
import pytorch_lightning as pl
from Classification.src.sider_gnn import SIDERGraphDataset, MPNN_SIDER

sider_dataset = SIDERGraphDataset(root='./sider_pyg', df=df_cleaned)

# Split the dataset
# We split the list of graph objects, not X and y matrices.
train_data, test_data = train_test_split(
    sider_dataset, train_size=0.8, random_state=0, shuffle=True
)
train_data, val_data = train_test_split(
    train_data, train_size=0.9, random_state=42, shuffle=True # 10% of train for validation
)

# DataLoaders handle batching of graphs for the model
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [ ]:
from Classification.src.sider_gnn_bayesian_opti import objective

import optuna

!wandb login

# Minimize the validation loss
study = optuna.create_study(direction="minimize")
study.optimize(
    lambda trial: objective(trial, train_loader, val_loader),
    n_trials=10)
# A trial = one test with a single combination of hyperparameters by training your GNN for the maximum of epochs

print("\n--- Bayesian Optimization Complete ---")
print(f"Best validation loss: {study.best_value:.4f}")
print("Best hyperparameters found:", study.best_params)

In [ ]:
completed_trials = len([t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE])

if completed_trials > 0:
    print(f"{completed_trials} trials were completed before stopping.")

    print("\nBest hyperparameters found so far:")
    print(study.best_params)

    print(f"\nBest validation loss so far: {study.best_value:.4f}")
else:
    print("No trials were completed before stopping.")

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint

best_params = study.best_params
print("Best hyperparameters found by Bayesian Optimization:")
print(best_params)

final_gnn_model = MPNN_SIDER(
    hidden_dim=best_params['hidden_dim'],
    out_dim=27,
    lr=best_params['lr']
)

# save the best version of this final model
final_checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    dirpath="final_model_checkpoints/",
    filename="best-final-gnn-model",
    save_top_k=1,
    mode="min"
)

final_trainer = pl.Trainer(
    max_epochs=50,
    callbacks=[final_checkpoint_callback],
    accelerator="auto"
)

print("\n--- Training the final GNN model with the best hyperparameters... ---")

# Train the final model
final_trainer.fit(model=final_gnn_model, train_dataloaders=train_loader, val_dataloaders=val_loader)

print("\n--- Final evaluation on the test set... ---")
final_trainer.test(model=final_gnn_model, dataloaders=test_loader, ckpt_path="best")

In [ ]:
checkpoint_path = "final_model_checkpoints/best-final-gnn-model.ckpt"

best_model_from_checkpoint = MPNN_SIDER.load_from_checkpoint(
    checkpoint_path,
    hidden_dim=best_params['hidden_dim'], # Use the value found from optimization
    out_dim=27
)

test_trainer = pl.Trainer()
test_trainer.test(model=best_model_from_checkpoint, dataloaders=test_loader)

## Use a pre-trained Transformer

In [ ]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel
import pandas as pd
from tqdm import tqdm

def get_chemberta_embeddings(smiles_list):
    """
    Generates embeddings for a list of SMILES strings using a pretrained ChemBERTa model.
    """
    tokenizer = AutoTokenizer.from_pretrained("seyonec/ChemBERTa-zinc-base-v1")
    model = AutoModel.from_pretrained("seyonec/ChemBERTa-zinc-base-v1")

    model.eval()

    all_embeddings = []

    for smiles in tqdm(smiles_list):
        inputs = tokenizer(smiles, return_tensors="pt", padding=True, truncation=True, max_length=128)

        # Get embeddings without calculating gradients for faster inference
        with torch.no_grad():
            outputs = model(**inputs)

        # Use the representation of the [CLS] token, which summarizes the entire molecule
        embedding = outputs.last_hidden_state[0, 0, :].numpy()
        all_embeddings.append(embedding)

    return np.array(all_embeddings)

In [ ]:
X_transformer_features = get_chemberta_embeddings(df_cleaned['canonical_smiles'].tolist())

In [ ]:
print(f"Shape of the feature matrix (X): {X_transformer_features.shape}")

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import hamming_loss, classification_report, roc_auc_score

y = df_final.iloc[:, 2:29]
X_train, X_test, y_train, y_test = train_test_split(X_transformer_features, y, test_size=0.2, random_state=42)

# Choose the best parameters from grid search
base_classifier = RandomForestClassifier(max_depth=50, n_estimators=100, random_state=42, class_weight='balanced')

multi_label_model = MultiOutputClassifier(estimator=base_classifier)

print("Training the multi-label model")
multi_label_model.fit(X_train, y_train)

# 1. Get the list of probability arrays
y_pred_proba_list = multi_label_model.predict_proba(X_test)

# 2. Reshape the probabilities into a proper 2D DataFrame
# We iterate through the list and extract the probability of the positive class (column at index 1) for each label.
y_pred_proba_df = pd.DataFrame({
    y_test.columns[i]: proba[:, 1]
    for i, proba in enumerate(y_pred_proba_list)
})

# 3. Calculate metrics
predictions = multi_label_model.predict(X_test)
loss = hamming_loss(y_test, predictions)

# Use the correctly formatted probabilities DataFrame for roc_auc_score
roc_auc_macro = roc_auc_score(y_test, y_pred_proba_df, average='macro')
roc_auc_micro = roc_auc_score(y_test, y_pred_proba_df, average='micro')

print(f"\nHamming Loss: {loss:.4f}")
print(f"ROC AUC Score (Macro Average): {roc_auc_macro:.4f}")
print(f"ROC AUC Score (Micro Average): {roc_auc_micro:.4f}")

In [ ]:
print(classification_report(y_test, predictions, target_names=y.columns))

## Multi-Layer Perceptron (MLP)

In [ ]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader, random_split
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import roc_auc_score
import numpy as np


class SIDERDataset(Dataset):
    """PyTorch Dataset wrapper for SIDER data."""
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)
        self.y = torch.FloatTensor(y.values if hasattr(y, 'values') else y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


class MLP_SIDER(pl.LightningModule):
    """
    Multi-Layer Perceptron for SIDER multi-label classification.
    
    Args:
        input_dim: Number of input features
        hidden_dims: List of hidden layer dimensions
        out_dim: Number of output labels (27 for SIDER)
        dropout: Dropout probability for regularization
        lr: Learning rate
    """
    
    def __init__(self, input_dim, hidden_dims=[512, 256, 128], out_dim=27, dropout=0.3, lr=0.001):
        super().__init__()
        self.save_hyperparameters()
        self.lr = lr
        
        # Build MLP architecture dynamically
        layers = []
        prev_dim = input_dim
        
        # Add hidden layers with activation, batch norm, and dropout
        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.BatchNorm1d(hidden_dim))
            layers.append(nn.Dropout(dropout))
            prev_dim = hidden_dim
        
        # Output layer (no activation - using BCEWithLogitsLoss)
        layers.append(nn.Linear(prev_dim, out_dim))
        
        self.network = nn.Sequential(*layers)
        
        # Loss function for multi-label classification
        self.criterion = nn.BCEWithLogitsLoss()
        
    def forward(self, x):
        return self.network(x)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        self.log('train_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.criterion(logits, y)
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        return loss
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        probs = torch.sigmoid(logits)
        return {'preds': probs, 'targets': y}
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.5, patience=5
        )
        return {
            'optimizer': optimizer,
            'lr_scheduler': {
                'scheduler': scheduler,
                'monitor': 'val_loss'
            }
        }

In [ ]:
# Prepare data for MLP
# Using the same split as before (X_train_scaled, X_test_scaled from cell 15)

train_dataset = SIDERDataset(X_train_scaled, y_train)
test_dataset = SIDERDataset(X_test_scaled, y_test)

# Create validation split from training data
train_size = int(0.9 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_subset, val_subset = random_split(
    train_dataset, [train_size, val_size],
    generator=torch.Generator().manual_seed(42)
)

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Training samples: {len(train_subset)}")
print(f"Validation samples: {len(val_subset)}")
print(f"Test samples: {len(test_dataset)}")
print(f"Input features: {X_train_scaled.shape[1]}")

In [ ]:
# Initialize MLP model
input_dim = X_train_scaled.shape[1]
mlp_model = MLP_SIDER(
    input_dim=input_dim,
    hidden_dims=[512, 256, 128],
    out_dim=27,
    dropout=0.3,
    lr=0.001
)

# Setup callbacks
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath='mlp_checkpoints/',
    filename='best-mlp-model',
    save_top_k=1,
    mode='min'
)

early_stop_callback = EarlyStopping(
    monitor='val_loss',
    patience=15,
    mode='min',
    verbose=True
)

# Train the model
trainer = pl.Trainer(
    max_epochs=100,
    callbacks=[checkpoint_callback, early_stop_callback],
    accelerator='auto',
    log_every_n_steps=10
)

print("--- Training MLP model... ---")
trainer.fit(mlp_model, train_loader, val_loader)

In [ ]:
# Evaluate on test set
print("\n--- Evaluating MLP on test set... ---")

mlp_model.eval()
all_preds = []
all_targets = []

with torch.no_grad():
    for batch in test_loader:
        x, y = batch
        logits = mlp_model(x)
        probs = torch.sigmoid(logits)
        all_preds.append(probs.cpu().numpy())
        all_targets.append(y.cpu().numpy())

all_preds = np.vstack(all_preds)
all_targets = np.vstack(all_targets)

# Calculate macro AUC-ROC
macro_auc = roc_auc_score(all_targets, all_preds, average='macro')
micro_auc = roc_auc_score(all_targets, all_preds, average='micro')

print(f"\nMLP Performance:")
print(f"Macro AUC-ROC: {macro_auc:.4f}")
print(f"Micro AUC-ROC: {micro_auc:.4f}")